In [6]:
# https://www.edureka.co/blog/web-scraping-with-python/
# https://stackoverflow.com/questions/34519746/using-python-docx-to-update-cell-content-of-a-table

from selenium import webdriver
from bs4 import BeautifulSoup
# import pandas as pd
from docx import Document, enum
from docx.shared import Pt

doc_name = "Vorlage_S.25.docx"
start_row_club_goalgetter_1 = 1
start_row_club_goalgetter_2 = 16
rows_available = 14

club_goalgetter = [
{"url": "https://www.fupa.net/team/sv-ohu-ahrain-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 0 },              
{"url": "https://www.fupa.net/team/sv-mengkofen-m1-2022-23/scorers",  "start_row": start_row_club_goalgetter_2, "start_column": 0 },              
{"url": "https://www.fupa.net/team/sg-rottenburg-oberhatzko-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 2 },
{"url": "https://www.fupa.net/team/tv-schierling-m2-2022-23/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 2 },             
{"url": "https://www.fupa.net/team/sg-adlhausen-langquaid-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 4 },
{"url": "https://www.fupa.net/team/sv-wallkofen-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 4 },
{"url": "https://www.fupa.net/team/sv-eggmuehl-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 6 },        
{"url": "https://www.fupa.net/team/fc-ottering-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 6 },       
{"url": "https://www.fupa.net/team/sv-essenbach-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 8 },   
{"url": "https://www.fupa.net/team/sc-postau-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 8 },  
{"url": "https://www.fupa.net/team/tsv-hofkirchen-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 10 }, 
{"url": "https://www.fupa.net/team/tsv-pfaffenberg-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_2, "start_column": 10 }, 
{"url": "https://www.fupa.net/team/sv-ettenkofen-m1-2022-23/scorers", "start_row": start_row_club_goalgetter_1, "start_column": 12 } ]

goalgetter_url = "https://www.fupa.net/league/kreisklasse-mallersdorf/scorers"

def fix_club_name(name):
    if name == "Pfaffenberg":
        name = "TSV Pfaffenberg"
    if name == "Mengkofen":
        name = "SV Mengkofen"
    if name == "Hofkirchen":
        name = "TSV Hofkirchen"
    if name == "Ettenkofen":
        name = "SV Ettenkofen"
    if name == "Ohu-Ahrain":
        name = "SV Ohu-Ahrain"
    if name == "Eggmühl":
        name = "SV Eggmühl"
    if name == "SG Rottenburg":
        name = "SG Rottenburg/Oberhatzkofen"
    if name == "Essenbach":
        name = "SV Essenbach"
    if name == "Wallkofen":
        name = "SV Wallkofen"
    if name == "Ottering":
        name = "FC Ottering"
    if name == "Postau":
        name = "SC Postau"
    if name == "Schierling":
        name = "TV Schierling II"
    # TODO: potential bug - need to check if Adlhausen scores a goal
    if name == "SG Adlhausen":
        name = "SG Adlhausen/Langquaid"
    return name

def set_font(doc, font_name):
    style = doc.styles['Normal']
    font = style.font
    font.name = font_name

def update_goalgetter(doc):
    driver = webdriver.Chrome()
    driver.get(goalgetter_url)
    content = driver.page_source
    soup = BeautifulSoup(content)
    rows = soup.find_all("div", {"class": "sc-1mrugnb-15 iOgjgp"})

    position, name, club, goals, assists, games = [], [], [], [], [], []

    for r in rows:
        position.append(r.find("div", {"class": "sc-1mrugnb-12 jjBzud"}).text)
        name.append(r.find("span", {"class": "sc-lhxcmh-0 jOiTFY"}).text)
        club.append(fix_club_name(r.find("span", {"class": "sc-lhxcmh-0 hudpXq"}).text))
        statistics = r.find_all("div", {"class": "sc-1mrugnb-12 itMjAW"})
        goals.append(statistics[0].text)
        assists.append(statistics[1].text)
        games.append(statistics[2].text)

    table = doc.tables[1]

    for x in range(19):
        # skip empty rows
        if x % 2 == 0:
            for y in range(7):
                if y == 1 :
                    table.cell(x,y).text = name[int(x/2)]
                elif y == 2:
                    table.cell(x,y).text = club[int(x/2)]
                elif y == 3:
                    table.cell(x,y).text = goals[int(x/2)]
                elif y == 5:
                    table.cell(x,y).text = assists[int(x/2)]
                table.cell(x,y).paragraphs[0].runs[0].font.size = Pt(8)
                table.cell(x,y).paragraphs[0].runs[0].font.bold = True

def update_club_goalgetter(doc):
    driver = webdriver.Chrome()
    table = doc.tables[0]

    # for i in range(2):
    for i in range(len(club_goalgetter)):
        driver.get(club_goalgetter[i]["url"])
        content = driver.page_source
        soup = BeautifulSoup(content)
        rows = soup.find_all("div", {"class": "sc-1mrugnb-15 iOgjgp"})
        name, goals = [], []

        for r in rows:
            name.append(r.find("span", {"class": "sc-lhxcmh-0 jOiTFY"}).text)
            statistics = r.find_all("div", {"class": "sc-1mrugnb-12 itMjAW"})
            goals.append(statistics[0].text)

        if len(name) > rows_available:
            print("length of name array: ", len(name), " and rows_available: ", rows_available)
            raise Exception("Not enough rows available, you need to add more rows!")
        
        for j in range(len(name)):
            cell_name = table.cell(club_goalgetter[i]["start_row"] + j, club_goalgetter[i]["start_column"])
            cell_name.text = name[j]
            cell_name.paragraphs[0].runs[0].font.size = Pt(6)
            cell_goals = table.cell(club_goalgetter[i]["start_row"] + j, club_goalgetter[i]["start_column"] + 1)
            cell_goals.text = goals[j]
            cell_goals.paragraphs[0].runs[0].font.size = Pt(6)
            cell_goals.paragraphs[0].alignment = enum.text.WD_ALIGN_PARAGRAPH.CENTER
            print("Name: ", name[j], " with goals: ", goals[j])

def init():
    doc = Document(doc_name)
    set_font(doc, 'Arial')
    update_club_goalgetter(doc)
    update_goalgetter(doc)
    doc.save("_S.25.docx")

init()


Name:  M. Malewski  with goals:  6
Name:  K. John  with goals:  5
Name:  P. Haseneder  with goals:  4
Name:  T. Haseneder  with goals:  4
Name:  L. Kölnberger  with goals:  3
Name:  M. Schorn  with goals:  2
Name:  I. Fahmy  with goals:  1
Name:  M. Zierer  with goals:  1
Name:  M. Rampeltshammer  with goals:  1
Name:  M. Zollner  with goals:  10
Name:  A. Altweck  with goals:  5
Name:  J. Zollner  with goals:  5
Name:  A. Gedrat  with goals:  3
Name:  S. Sattler  with goals:  3
Name:  M. Sattler  with goals:  2
Name:  P. Edenhofer  with goals:  1
Name:  J. Steubl  with goals:  1
Name:  T. Köglmeier  with goals:  1
Name:  D. Schwarz  with goals:  1
Name:  P. Fritsch  with goals:  1
Name:  G. Satici  with goals:  5
Name:  L. Matriciano  with goals:  3
Name:  M. Hofmeister  with goals:  2
Name:  G. Matriciano  with goals:  2
Name:  M. Blaimer  with goals:  1
Name:  D. Köhler  with goals:  1
Name:  L. Sponbrucker  with goals:  1
Name:  M. Hüttner  with goals:  1
Name:  L. Kuntze  with goa